In [3]:
import utils

# Login to ECR

In [1]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 654654589924.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/sagemaker-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credential-stores

Login Succeeded


# Checking current images

In [2]:
!docker images

REPOSITORY   TAG       IMAGE ID   CREATED   SIZE


# Creating ECR repository

In [4]:
user = utils.get_username()
repository_name = f"utec-mlops/{user}/online-prediction"
repository_name

'utec-mlops/ricardo-puma/online-prediction'

In [5]:
!aws ecr create-repository --repository-name $repository_name

{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-east-1:654654589924:repository/utec-mlops/ricardo-puma/online-prediction",
        "registryId": "654654589924",
        "repositoryName": "utec-mlops/ricardo-puma/online-prediction",
        "repositoryUri": "654654589924.dkr.ecr.us-east-1.amazonaws.com/utec-mlops/ricardo-puma/online-prediction",
        "createdAt": "2025-06-10T21:21:20.048000+00:00",
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}


# Build image

In [6]:
!docker build --network sagemaker -t $repository_name .

DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            BuildKit is currently disabled; enable it by removing the DOCKER_BUILDKIT=0
            environment-variable.

Sending build context to Docker daemon    148kB
Step 1/5 : FROM public.ecr.aws/lambda/python:3.8
3.8: Pulling from lambda/python

bc7afcc5: Pulling fs layer 
885bc8e5: Pulling fs layer 
3b657f0b: Pulling fs layer 
b66d8510: Pulling fs layer 
b7ce71c1: Pulling fs layer 
Digest: sha256:bb3102c119726c2dc5a08ddf773f1e75499555e07f56a831ab2e3a097f6ef46d
Status: Downloaded newer image for public.ecr.aws/lambda/python:3.8
 ---> cfe1181eef94
Step 2/5 : RUN pip install --no-cache-dir mlflow==2.13.2 sagemaker-mlflow==0.1.0 xgboost==2.1.4 boto3 numpy pandas
 ---> Running in 88c6718adc81
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.0/25.0 MB 247.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 198.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9

# Checking current images

In [7]:
!docker images

REPOSITORY                                  TAG       IMAGE ID       CREATED         SIZE
utec-mlops/ricardo-puma/online-prediction   latest    c93a2a48f35a   3 minutes ago   1.25GB


# Tag image

In [12]:
ecr_url = "654654589924.dkr.ecr.us-east-1.amazonaws.com"
tag_name = f"{repository_name}:latest"
full_tag_name = f"{ecr_url}/{tag_name}"
print(tag_name)
print(full_tag_name)

utec-mlops/ricardo-puma/online-prediction:latest
654654589924.dkr.ecr.us-east-1.amazonaws.com/utec-mlops/ricardo-puma/online-prediction:latest


In [13]:
!docker tag $tag_name $full_tag_name

# Checking current images

In [17]:
!docker images

REPOSITORY                                                                               TAG       IMAGE ID       CREATED          SIZE
654654589924.dkr.ecr.us-east-1.amazonaws.com/utec-mlops/ricardo-puma/online-prediction   latest    c93a2a48f35a   12 minutes ago   1.25GB
utec-mlops/ricardo-puma/online-prediction                                                latest    c93a2a48f35a   12 minutes ago   1.25GB


# Push image to ECR

In [19]:
!docker push $full_tag_name

The push refers to repository [654654589924.dkr.ecr.us-east-1.amazonaws.com/utec-mlops/ricardo-puma/online-prediction]

b07f24a9: Preparing 
047a4bd9: Preparing 
76196de3: Preparing 
541d45e3: Preparing 
ef616f15: Preparing 
fc379876: Preparing 
03d018ef: Preparing 
latest: digest: sha256:6b6ffd0e390a4e60eb294fbf046f12c74b22ee16abeff02f3365cced044f41df size: 2001


# Create Lambda Function

In [41]:
function_name = f"mlops-utec-{user}-online-prediction"
print(function_name)
lambda_role = "arn:aws:iam::654654589924:role/utec-lambda-mlops-role"

mlops-utec-ricardo-puma-online-prediction


In [42]:
!aws lambda create-function \
        --function-name $function_name \
        --package-type Image \
        --code ImageUri=$full_tag_name \
        --role $lambda_role \
        --architectures x86_64 \
        --region us-east-1 \
        --timeout 120

{
    "FunctionName": "mlops-utec-ricardo-puma-online-prediction",
    "FunctionArn": "arn:aws:lambda:us-east-1:654654589924:function:mlops-utec-ricardo-puma-online-prediction",
    "Role": "arn:aws:iam::654654589924:role/utec-lambda-mlops-role",
    "CodeSize": 0,
    "Description": "",
    "Timeout": 120,
    "MemorySize": 128,
    "LastModified": "2025-06-10T22:40:07.591+0000",
    "CodeSha256": "6b6ffd0e390a4e60eb294fbf046f12c74b22ee16abeff02f3365cced044f41df",
    "Version": "$LATEST",
    "TracingConfig": {
        "Mode": "PassThrough"
    },
    "RevisionId": "2cd86dad-f90a-4985-9b47-1ea71c0460ec",
    "State": "Pending",
    "StateReason": "The function is being created.",
    "StateReasonCode": "Creating",
    "PackageType": "Image",
    "Architectures": [
        "x86_64"
    ],
    "EphemeralStorage": {
        "Size": 512
    },
    "SnapStart": {
        "ApplyOn": "None",
        "OptimizationStatus": "Off"
    },
    "LoggingConfig": {
        "LogFormat": "Text",
     

In [ ]:
# Create a test "prediction-test" and add this input
{
  "body": {
    "data": [0,237.10,487.97,504.7,7878.1]
  }
}

# Delete Image

In [ ]:
!docker rmi <IMAGE_ID>   
!docker rmi <FULL_TAG_NAME>